In [1]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 49.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.8 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
google-spark-connect 0.5.2 requires google-api-core>=2.19.1, but you have google-api-core 1.34.1 which is incompatible.
google-cloud-bigtable 2.30.0 requires google-api-core[grpc]<3.0.0,>=2.16.0, but you have google

In [2]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import os
from tqdm import tqdm
from PIL import Image
import random
import torchvision.transforms.functional as TransformerF
import torch.nn as nn
from torchvision import transforms, models
import torch.nn.functional as F
import math
from copy import deepcopy
import torchvision.transforms as T
import time
import shutil
import kagglehub



# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh

2025-07-05 08:34:47.106643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751704487.321714      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751704487.383690      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
model_path = kagglehub.model_download("v7wedv/91-accuracy-multi-module-fusion-model/pyTorch/default")
model_path = os.path.join(model_path,"checkpoint.pth")
boundaryModelPath = kagglehub.dataset_download("ggez1244/boundary-model-weights")
boundaryModelPath = os.path.join(boundaryModelPath,"boundary_model_weights (1).pth")

In [4]:
class PositionalEmbedding(nn.Module):
    def __init__(self):
        super(PositionalEmbedding, self).__init__()
        self.pos_embed = nn.Embedding(32, 2048)

    def forward(self, x):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, device=x.device).unsqueeze(0).repeat(x.size(0), 1)
        pos_embed = self.pos_embed(pos)
        return x + pos_embed



class CrossAttention(nn.Module):
    def __init__(self, image_dim, pose_dim, project_dim, num_heads=4):
        super(CrossAttention, self).__init__()
        self.image_pre_norm = nn.LayerNorm(normalized_shape=image_dim)
        self.pose_pre_norm = nn.LayerNorm(normalized_shape=pose_dim)

        self.pose_project = nn.Linear(pose_dim, project_dim)
        self.image_project = nn.Linear(image_dim, project_dim)

        self.image_self_attention= nn.MultiheadAttention(embed_dim=project_dim, num_heads=num_heads, batch_first=True)
        self.pose_self_Attenstion = nn.MultiheadAttention(embed_dim=project_dim, num_heads=num_heads, batch_first=True)

        self.image_layer_norm = nn.LayerNorm(normalized_shape=project_dim)
        self.pose_layer_norm = nn.LayerNorm(normalized_shape=project_dim)
        # self.output_layer_norm = nn.LayerNorm(normalized_shape=int(project_dim*2))


        self.cross_attention1 = nn.MultiheadAttention(embed_dim=project_dim, num_heads=num_heads, batch_first=True)
        self.cross_attention2 = nn.MultiheadAttention(embed_dim=project_dim, num_heads=num_heads, batch_first=True)


    def forward(self, image_input, pose_input):
        image_features = self.image_pre_norm(image_input)
        pose_features = self.pose_pre_norm(pose_input)

        pose_projection = self.pose_project(pose_features)
        image_projection = self.image_project(image_features)

        pose_projection_attention, _ = self.pose_self_Attenstion(query=pose_projection, key=pose_projection, value=pose_projection)
        image_projection_attention, _ = self.image_self_attention(query=image_projection, key=image_projection, value=image_projection)

        residual_image_connection = image_projection+image_projection_attention
        residual_pose_connection = pose_projection+pose_projection_attention

        image = self.image_layer_norm(residual_image_connection)
        pose = self.pose_layer_norm(residual_pose_connection)

        attn_output1, _ = self.cross_attention1(query=image, key=pose, value=pose)
        attn_output2, _ = self.cross_attention2(query=pose, key=image, value=image)

        # output = self.final_feedforward_layer(torch.cat([ attn_output1, attn_output2 ],dim=2))

        # output = self.output_layer_norm(output)

        return torch.cat([ attn_output1, attn_output2 ],dim=2)

class Model(nn.Module):
    def __init__(self,embedding = 5):
        super().__init__()

        self.crossAttentionLayer = CrossAttention(image_dim=6320, pose_dim=3102,project_dim=1024, num_heads=4)


        mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
        mobilenet_features = mobilenet.features

        t=0
        for layer in mobilenet.parameters():
            t+=1

        for i,layer in enumerate(mobilenet.parameters()):
            layer.requires_grad = False
            if i== (t-4) or i== (t-5):
              layer.requires_grad = True

        self.cropedImageCNN = nn.Sequential(
                mobilenet_features,

                nn.Conv2d(1280, 1580, kernel_size=3),
                nn.BatchNorm2d(1580),
                nn.SiLU(),
                nn.AvgPool2d(kernel_size=2),

                nn.Conv2d(1580, 1580, kernel_size=3),
                nn.BatchNorm2d(1580),
                nn.SiLU(),
                nn.AvgPool2d(kernel_size=2),

                nn.Flatten()
            )


        self.leftHandCNN = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 40x40

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 20x20

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 10x10

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 5x5
            nn.Flatten(),
            nn.Linear(256 * 5 * 5, 1280),

        )

        self.rightHandCNN = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 40x40

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 20x20

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 10x10

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 5x5
            nn.Flatten(),
            nn.Linear(256 * 5 * 5, 1280),

        )

        self.leftHandVRegionEmbedding = nn.Sequential(
            nn.Linear(1,embedding),
        )
        self.leftHandHRegionEmbedding = nn.Sequential(
            nn.Linear(1,embedding),
        )
        self.rightHandVRegionEmbedding = nn.Sequential(
            nn.Linear(1,embedding),
        )
        self.rightHandHRegionEmbedding = nn.Sequential(
            nn.Linear(1,embedding),
        )
        self.leftHandDistance = nn.Sequential(
            nn.Linear(2,embedding),
        )
        self.rightHandDistance = nn.Sequential(
            nn.Linear(2,embedding),
        )
        self.landmarksEmbedding = nn.Sequential(
            nn.Linear(1106 ,512),
        )

        encoder_layer = nn.TransformerEncoderLayer(
                d_model=2048,
                nhead=8,
                dim_feedforward=3072,
                dropout=0.0,
                activation='gelu',
                batch_first=True
            )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)


        self.positionalEmbedding = PositionalEmbedding()

        self.classifier = nn.Sequential(
            nn.Linear(2048, 26),
        )
        self.landmarksLayerNorm = nn.LayerNorm(normalized_shape=1106)
        # self.dropout1 = nn.Dropout(0.05)
        # self.dropout2 = nn.Dropout(0.05)

    def forward(self,x,croped_image, train= False):
        batch_size,frames, features = x.size()

        # print(x.size())

        # print(x.size())

        LHI,RHI,LHVR,RHVR,LHHR,RHHR,LHD,RHD,ALL_LANDMARKS =x[:,:,:19200]/255,x[:,:,19200:38400]/255,x[:,:,38400],x[:,:,38401],x[:,:,38402],x[:,:,38403],x[:,:,38404:38406],x[:,:,38406:38408],x[:,:,38408:]
        LHI = LHI.reshape(batch_size,frames,80,80,3).float()
        LHI = LHI.reshape(batch_size*frames, 80,80,3)
        LHI = LHI.permute(0,3,1,2)
        LHI_output = self.leftHandCNN(LHI)
        LHI_output = LHI_output.reshape(batch_size,frames,-1) #1280

        RHI = RHI.reshape(batch_size,frames,80,80,3).float()
        RHI = RHI.reshape(batch_size*frames, 80,80,3)
        RHI = RHI.permute(0,3,1,2)
        # print(RHI.shape)
        RHI_output = self.rightHandCNN(RHI)
        RHI_output = RHI_output.reshape(batch_size,frames,-1) #1280

        # print('left hand regoin',LHR.shape)
        LHVR_output = self.leftHandVRegionEmbedding(LHVR.unsqueeze(-1).float())
        LHHR_output = self.leftHandHRegionEmbedding(LHHR.unsqueeze(-1).float())
        RHVR_output = self.rightHandVRegionEmbedding(RHVR.unsqueeze(-1).float())
        RHHR_output = self.rightHandHRegionEmbedding(RHHR.unsqueeze(-1).float())

        LHD_output = self.leftHandDistance(LHD.float())
        RHD_output = self.rightHandDistance(RHD.float())

        all_landmarks_output = self.landmarksEmbedding(self.landmarksLayerNorm(ALL_LANDMARKS.float()))

        batch_size,frames,c,w,h = croped_image.shape
        croped_images_input = croped_image.reshape(batch_size*frames,c,w,h)

        croped_output = self.cropedImageCNN(croped_images_input/255)#[128, 512, 3, 3]

        croped_output = croped_output.reshape(batch_size,frames,-1)
        # croped_output = self.dropout1(croped_output)
        # print(croped_output.shape)
        flattend_output = torch.cat([LHI_output, RHI_output,LHVR_output,LHHR_output, RHVR_output,RHHR_output,LHD_output, RHD_output, all_landmarks_output],dim=2)
        # flattend_output = self.dropout2(flattend_output)
        # flattend_output = self.dropout(flattend_output)
        combined_output = self.crossAttentionLayer(croped_output,flattend_output)
        combined_output = self.positionalEmbedding(combined_output)

        transformer_output = self.transformer(combined_output)

        global_average_pooling = transformer_output.mean(dim=1)


        output = self.classifier(global_average_pooling)


        return output


In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-torch.log(torch.tensor(10000.0)) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].to(x.device)
        return x


class BoundaryDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.src_projection = nn.Sequential(
            nn.LayerNorm(33*2),
            nn.Linear(33*2,128),
            nn.Linear(128,256)
        )
        encoder_layer = nn.TransformerEncoderLayer(
                d_model=256,
                nhead=4,
                dim_feedforward=512,
                dropout=0.1,
                activation='gelu',
                batch_first=True
            )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=6)


        self.positionalEmbedding = PositionalEncoding(256)
        self.output_layer = nn.Sequential(
            nn.Linear(256,128),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(128,2)
        )

    def forward(self,landmarks):
        batch_size,seq_len, _,_ = landmarks.size()
        landmarks = landmarks.reshape(batch_size, seq_len,-1)
        landmarks = self.src_projection(landmarks)

        landmarks = self.positionalEmbedding(landmarks)

        transformer_out = self.transformer(landmarks)
        logits = self.output_layer(transformer_out)
        return logits


In [6]:
class Extractor():
  def __init__(self,debuging,target_size):
    self.BG_COLOR = (0, 0, 0)
    self.debuging = debuging
    self.pose = mp_pose.Pose(
        static_image_mode=True,
        model_complexity=0,
        enable_segmentation=True,
        min_detection_confidence=0.1
        )

    self.hands = mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.1
        )

    self.face_mesh = mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.1
       )


  def extractLandmarks(self, video):
    if len(video.shape)==4:
        landmarks= []
        segmented_images = []
        images = []
        for image in tqdm(video):
          info = {}
          pose_landmarks = self.extractPose(image)
          hands_landmarks = self.extractHands(image)
          face_landmarks = self.extractFace(image)

          if not pose_landmarks["pose_landmark"]:
            info["pose_landmarks"] = None
            info["body_segmentation"] = pose_landmarks["body_segmentation"]
          else:
            info["pose_landmarks"] = pose_landmarks["pose_landmark"]
            info["body_segmentation"] = pose_landmarks["body_segmentation"]

          if not hands_landmarks:
            info["hands_landmarks"] = {
                "right_hand":None,
                "left_hand":None,
                "landmarks":None
            }
          else:
            info["hands_landmarks"] = hands_landmarks

          if not face_landmarks:
            info["face_landmarks"] = None
          else:
            info["face_landmarks"] = face_landmarks

          checker = self.check(info)
          x_y_landmarks = self.get_x_y_landmarks_scaled_up(info,image)

          right_hand = 0
          left_hand = 0
          face = 0
          pose = 0
          if checker["right_hand_landmarks"]:
            right_hand = 1
          if checker["left_hand_landmarks"]:
            left_hand = 1
          if checker["pose_landmarks"]:
              pose = 1
          if checker["face_landmarks"]:
              face =1
          x_y_landmarks.append([right_hand,pose])
          x_y_landmarks.append([face,left_hand])
          landmarks.append(x_y_landmarks)
          segmented_images.append(self.getSegmentedImage(info,image))
          images.append(image)
          if self.debuging:
              print(checker)
              self.showLandmarks(info,image)
        return landmarks, segmented_images, images
    else:
          image = video
          info = {}
          pose_landmarks = self.extractPose(image)
          hands_landmarks = self.extractHands(image)
          face_landmarks = self.extractFace(image)

          if not pose_landmarks["pose_landmark"]:
            info["pose_landmarks"] = None
            info["body_segmentation"] = pose_landmarks["body_segmentation"]
          else:
            info["pose_landmarks"] = pose_landmarks["pose_landmark"]
            info["body_segmentation"] = pose_landmarks["body_segmentation"]

          if not hands_landmarks:
            info["hands_landmarks"] = {
                "right_hand":None,
                "left_hand":None,
                "landmarks":None
            }
          else:
            info["hands_landmarks"] = hands_landmarks

          if not face_landmarks:
            info["face_landmarks"] = None
          else:
            info["face_landmarks"] = face_landmarks

          checker = self.check(info)
          return checker


  def get_x_y_landmarks_scaled_up(self,info,image):
    landmarks = self.get_x_y_landmarks(info,image)
    shape = image.shape
    landmarks = [[t[0]*shape[1],t[1]*shape[0]] for t in landmarks]
    return landmarks
  def get_x_y_landmarks(self,info, image):

    x_y_landmarks = []
    if info["hands_landmarks"]["right_hand"] != None:
      for landmark in info["hands_landmarks"]["right_hand"].landmark:
        x_y_landmarks.append([landmark.x,landmark.y])
    else:
      x_y_landmarks.extend(list(np.zeros((21,2))))


    if info["pose_landmarks"] != None:
      for landmark in info["pose_landmarks"].landmark:
        x_y_landmarks.append([landmark.x,landmark.y])
    else:
      x_y_landmarks.extend(list(np.zeros((33,2))))


    if info["face_landmarks"] != None:
      for landmark in info["face_landmarks"].landmark:
        x_y_landmarks.append([landmark.x,landmark.y])
    else:
      x_y_landmarks.extend(list(np.zeros((478,2))))


    if info["hands_landmarks"]["left_hand"] != None:
      for landmark in info["hands_landmarks"]["left_hand"].landmark:
        x_y_landmarks.append([landmark.x,landmark.y])
    else:
      x_y_landmarks.extend(list(np.zeros((21,2))))

    return x_y_landmarks

  def check(self,info):
    return {
        "pose_landmarks":info["pose_landmarks"] !=None,
        "right_hand_landmarks":info["hands_landmarks"]["right_hand"] !=None,
        "left_hand_landmarks":info["hands_landmarks"]["left_hand"] !=None,
        "face_landmarks":info["face_landmarks"]!=None
    }
  def extractPose(self,image):
      results = self.pose.process(image)
      if not results.pose_landmarks:
        return {
            "pose_landmark":None,
            "body_segmentation": results.segmentation_mask
            }
      return {
            "pose_landmark":results.pose_landmarks,
            "body_segmentation": results.segmentation_mask
            }


  def extractHands(self,image):
      results = self.hands.process(image)

      if not results.multi_hand_landmarks:
        return None
      data = {
          "right_hand":None,
          "left_hand":None,
          "landmarks":results.multi_hand_landmarks
          }
      for handedness, landmarks in zip(results.multi_handedness, results.multi_hand_landmarks):
          label = handedness.classification[0].label
          if label == "Right":
              data["right_hand"] = landmarks
          elif label == "Left":
              data["left_hand"] = landmarks
      return data


  def extractFace(self,image):
      results = self.face_mesh.process(image)

      if not results.multi_face_landmarks:
        return None
      return results.multi_face_landmarks[0]

  def showSegmentedBody(self):
    if self.debuging:
      condition = np.stack((self.info["body_segmentation"],) * 3, axis=-1) > 0.1
      bg_image = np.zeros(self.image.shape, dtype=np.uint8)
      bg_image[:] = self.BG_COLOR
      annotated_image = np.where(condition, self.image, bg_image)
      plt.imshow(annotated_image)
      plt.show()
    else:
      print("enable debuging")

  def getSegmentedImage(self,info,image):
      if  info['body_segmentation'] is not None:
          condition = np.stack((info["body_segmentation"],) * 3, axis=-1) > 0.1
          bg_image = np.zeros(image.shape, dtype=np.uint8)
          bg_image[:] = self.BG_COLOR
          annotated_image = np.where(condition, image, bg_image)
          return annotated_image
      else:
          print("there is no segmented image for this frame")
          return list(np.zeros((512,512,3)))

  def showLandmarks(self,info,image):
    image_with_landmarks = image.copy()
    if info["pose_landmarks"] != None:

      mp_drawing.draw_landmarks(
              image_with_landmarks,
              info["pose_landmarks"],
              mp_pose.POSE_CONNECTIONS,
              landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    if info["hands_landmarks"]['landmarks'] != None:
      for hand_landmarks in info["hands_landmarks"]['landmarks']:
        mp_drawing.draw_landmarks(
            image_with_landmarks,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

    if info["face_landmarks"] != None:
        mp_drawing.draw_landmarks(
            image=image_with_landmarks,
            landmark_list=info["face_landmarks"],
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
            image=image_with_landmarks,
            landmark_list=info["face_landmarks"],
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
            image=image_with_landmarks,
            landmark_list=info["face_landmarks"],
            connections=mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_iris_connections_style())
    plt.imshow(image_with_landmarks)
    plt.show()


In [7]:
class ImageCropper():
  def __init__(self,debuging,target_size = (512, 512) ):
    self.target_size=target_size
    self.margin=20
    self.debuging= debuging
    self.transform=None
    self.normalized=False
    if debuging:
      self.cropped_image = None

  def cropImage(self,image, landmarks, checker):
    if torch.is_tensor(image):
        image = image.cpu().numpy()
        image = (image).astype(np.uint8)
    if torch.is_tensor(landmarks):
        landmarks_np = landmarks.cpu().numpy()
    else:
        landmarks_np = np.array(landmarks).copy()

    # Ensure landmarks shape is (N, 2)
    if landmarks_np.ndim == 1 and landmarks_np.shape[0] % 2 == 0:
        landmarks_np = landmarks_np.reshape(-1, 2)
    elif landmarks_np.ndim != 2 or landmarks_np.shape[1] != 2:
        raise ValueError(f"Invalid landmarks shape: {landmarks_np.shape}")

    frame = image
    lm = landmarks_np
    if not checker['face_landmarks']:
        variable = lm[6+21]
        variable[1]-=50
        lm[54:532,:] = variable


    if not checker['right_hand_landmarks']:
        lm[:21,:] = lm[17+21]

    if not checker['left_hand_landmarks']:
        lm[532:,:] = lm[18+21]
    # print("this in cropped image")
    # for i in range(len(lm[21:54,:])):
    #       print("index: ",i,") pose: ",lm[21:54,:][i])

    cropped_image, (x_min, x_max, y_min, y_max) = self.detect_and_crop_person(frame, lm, margin=self.margin)
    if cropped_image.size == 0:
        adjusted_landmarks = np.zeros_like(landmarks_np)
        output_image = torch.zeros(3, self.target_size[1], self.target_size[0])
        return output_image, adjusted_landmarks

    processed_img, scale, (x_offset, y_offset) = self.resize_and_pad(cropped_image, target_size=self.target_size)

    if self.normalized:
        h, w, _ = frame.shape
        x_coords = lm[:, 0] * w
        y_coords = lm[:, 1] * h
    else:
        x_coords = lm[:, 0]
        y_coords = lm[:, 1]

    adjusted_x = (x_coords - x_min) * scale + x_offset
    adjusted_y = (y_coords - y_min) * scale + y_offset

    target_w, target_h = self.target_size
    # out_of_bounds = (
    #     (adjusted_x < 0) | (adjusted_x >= target_w) |
    #     (adjusted_y < 0) | (adjusted_y >= target_h)
    # )
    # adjusted_x[out_of_bounds] = 0.0
    # adjusted_y[out_of_bounds] = 0.0

    if self.normalized:
        adjusted_x /= target_w
        adjusted_y /= target_h

    adjusted_landmarks = np.stack([adjusted_x, adjusted_y], axis=-1)

    if self.transform:
        processed_img = self.transform(processed_img)
    else:
        processed_img = torch.from_numpy(processed_img).permute(2, 0, 1).float()
    if self.debuging:
      self.cropped_image = processed_img
    return processed_img, adjusted_landmarks

  def resize_and_pad(self,image, target_size=(512, 512)):
      target_w, target_h = target_size
      h, w, _ = image.shape
      if h == 0 or w == 0:
          return np.zeros((target_h, target_w, 3), dtype=np.uint8), 1.0, (0, 0)
      padded_image = np.zeros((target_h, target_w, 3), dtype=np.uint8)
      scale = min(target_w / w, target_h / h)
      new_w, new_h = int(w * scale), int(h * scale)
      if new_w == 0 or new_h == 0:
          return padded_image, 1.0, (0, 0)
      resized_image = cv2.resize(image, (new_w, new_h))
      x_offset = (target_w - new_w) // 2
      y_offset = (target_h - new_h) // 2
      padded_image[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized_image
      return padded_image, scale, (x_offset, y_offset)

  def detect_and_crop_person(self,frame, lm, margin=10):
      h, w, _ = frame.shape
      if self.normalized:
          x_coords = lm[:, 0] * w
          y_coords = lm[:, 1] * h
      else:
          x_coords = lm[:, 0]
          y_coords = lm[:, 1]
      x_min = int(max(np.min(x_coords) - margin, 0))
      x_max = int(min(np.max(x_coords) + margin, w))
      y_min = int(max(np.min(y_coords) - margin, 0))
      y_max = int(min(np.max(y_coords) + margin, h))
      if x_max <= x_min or y_max <= y_min:
          return frame, (0, w, 0, h)
      crop = frame[y_min:y_max, x_min:x_max]
      if crop.size == 0:
          return frame, (0, w, 0, h)
      return crop, (x_min, x_max, y_min, y_max)
  def showImage(self):
    print("this cropped image: ")
    image = self.cropped_image.permute(1,2,0).numpy().astype(dtype=np.uint16)
    plt.imshow(image)
    plt.show()

In [8]:
class FeaturesExtractorFromImage():
  def __init__(self,debuging):
    if debuging:
      self.info = {
          "left_hand_image":None,
          "right_hand_image":None,
          "vertical":None,
          "horizontal":None,
          "left_distance":None,
          "right_distance":None
      }

    self.debuging= debuging
    pass

  def extractFeatures(self,image, landmarks_checker, scaled_landmarks):

    if not landmarks_checker["pose_landmarks"] and not landmarks_checker["right_hand_landmarks"] and not landmarks_checker["left_hand_landmarks"] and not landmarks_checker["face_landmarks"]:
      return list(np.zeros((39514)))
    scaled_landmarks = scaled_landmarks
    # right hands landmarks:  21
    # pose landmarks:  33
    # face landmarks:  478
    # left hands landmarks:  21
    right_hand_landmarks,pose_landmarks,face_landmarks,left_hand_landmarks = scaled_landmarks[:21,:],scaled_landmarks[21:54,:],scaled_landmarks[54:532,:],scaled_landmarks[532:,:]
    # print("this now in mediapipe")
    # for i in range(len(scaled_landmarks[21:54,:])):
    #       print("index: ",i,") pose: ",scaled_landmarks[21:54,:][i])

    vertical_regoins, horizontal_regions, right_hands_box_cordinations, pose_box_cordinations, face_box_cordinations, left_hands_box_cordinations =self.handsRegoins(
      landmarks_checker,
      right_hand_landmarks,
      pose_landmarks,
      face_landmarks,
      left_hand_landmarks,
      image
      )

    left_distance,right_distance,left_hand_image, right_hand_image = self.getHandsDistanceFromFace(
      landmarks_checker,
      right_hands_box_cordinations,
      pose_box_cordinations,
      face_box_cordinations,
      left_hands_box_cordinations,
      image
      )

    LR_hand_regions = np.array([vertical_regoins,horizontal_regions])

    # print(left_hand_image.reshape(-1).shape)
    # print(right_hand_image.reshape(-1).shape)
    # print(LR_hand_regions.reshape(-1).shape)
    # print(left_distance.reshape(-1).shape)
    # print(right_distance.reshape(-1).shape)
    # print(right_hand_landmarks.reshape(-1).shape)
    # print(pose_landmarks.reshape(-1).shape)
    # print(face_landmarks.reshape(-1).shape)
    # print(left_hand_landmarks.reshape(-1).shape)

    # print("maximum inside extract functino: ",left_hand_image.max())
    # plt.imshow( left_hand_image)
    # plt.show()


    frame_features = np.concatenate([left_hand_image.reshape(-1),right_hand_image.reshape(-1),LR_hand_regions.reshape(-1),left_distance.reshape(-1),right_distance.reshape(-1), right_hand_landmarks.reshape(-1),pose_landmarks.reshape(-1),face_landmarks.reshape(-1),left_hand_landmarks.reshape(-1)])
    return torch.tensor(frame_features)

  def getHandsDistanceFromFace(self,landmarks_checker,right_hands_box_cordinations, pose_box_cordinations, face_box_cordinations, left_hands_box_cordinations, image):
      margin = 10

      left_hand_image= image[left_hands_box_cordinations[0][1]-margin:left_hands_box_cordinations[1][1]+margin,
                        left_hands_box_cordinations[0][0]-margin:left_hands_box_cordinations[1][0]+margin]

      right_hand_image= image[right_hands_box_cordinations[0][1]-margin:right_hands_box_cordinations[1][1]+margin,
                        right_hands_box_cordinations[0][0]-margin:right_hands_box_cordinations[1][0]+margin]
      left_hand_image = cv2.resize(left_hand_image, (80,80)) if left_hand_image.shape[0] > 0 and left_hand_image.shape[1] > 0 else np.zeros((80,80,3))
      right_hand_image = cv2.resize(right_hand_image, (80,80)) if right_hand_image.shape[0] > 0 and right_hand_image.shape[1] > 0 else np.zeros((80,80,3))
      distance_between_left_face = np.array([np.abs(face_box_cordinations[0][0]-left_hands_box_cordinations[0][0]),np.abs(face_box_cordinations[1][1] -left_hands_box_cordinations[1][1])])
      distance_between_right_face = np.array([np.abs(face_box_cordinations[0][0]-right_hands_box_cordinations[0][0]),np.abs(face_box_cordinations[1][1] -right_hands_box_cordinations[1][1])])
      if self.debuging:
        self.info['left_distance'] = distance_between_left_face
        self.info['right_distance'] = distance_between_right_face
        self.info['left_hand_image'] = left_hand_image
        self.info['right_hand_image'] = right_hand_image
      return distance_between_left_face,distance_between_right_face,(left_hand_image),(right_hand_image)


  def handsRegoins(self, landmarks_checker, right_hand_landmarks,pose_landmarks,face_landmarks,left_hand_landmarks,image):
    right_hands_box_cordinations = np.min(right_hand_landmarks,axis = 0).astype(int), np.max(right_hand_landmarks,axis=0).astype(int)#min bottom left - max top right
    pose_box_cordinations = np.min(pose_landmarks,axis = 0).astype(int), np.max(pose_landmarks,axis=0).astype(int)#min bottom left - max top right
    face_box_cordinations = np.min(face_landmarks,axis = 0).astype(int), np.max(face_landmarks,axis=0).astype(int)#min bottom left - max top right
    left_hands_box_cordinations = np.min(left_hand_landmarks,axis = 0).astype(int), np.max(left_hand_landmarks,axis=0).astype(int)#min bottom left - max top right

    if not landmarks_checker['face_landmarks']:
      new_box_cordinations = np.array([pose_landmarks[8],pose_landmarks[7],pose_landmarks[10],pose_landmarks[9]])
      face_box_cordinations = np.min(new_box_cordinations,axis = 0).astype(int),np.max(new_box_cordinations,axis=0).astype(int)

    if not landmarks_checker["right_hand_landmarks"]:
      new_box_cordinations = np.array([pose_landmarks[21],pose_landmarks[19],pose_landmarks[15],pose_landmarks[17]])
      right_hands_box_cordinations = np.min(new_box_cordinations,axis = 0).astype(int),np.max(new_box_cordinations,axis=0).astype(int)
    if not landmarks_checker["left_hand_landmarks"]:
      new_box_cordinations = np.array([pose_landmarks[20],pose_landmarks[22],pose_landmarks[18],pose_landmarks[16]])
      left_hands_box_cordinations = np.min(new_box_cordinations,axis = 0).astype(int),np.max(new_box_cordinations,axis=0).astype(int)

    vertical_hands_locations =[]
    for hand in [left_hands_box_cordinations,right_hands_box_cordinations]:
      if hand[0][1] <=face_box_cordinations[0][1]:
        vertical_hands_locations.append(1)
      elif hand[0][1] >=face_box_cordinations[0][1] and hand[0][1] < pose_box_cordinations[0][1]:
        vertical_hands_locations.append(2)
      elif hand[0][1] <face_box_cordinations[1][1] and hand[0][1] >= pose_box_cordinations[0][1]:
        vertical_hands_locations.append(3)
      elif hand[0][1] >pose_box_cordinations[0][1] and hand[0][1]>= face_box_cordinations[1][1] and hand[0][1]< (pose_box_cordinations[0][1]/2)+pose_box_cordinations[0][1] :
        vertical_hands_locations.append(4)
      else:
        vertical_hands_locations.append(5)
    if self.debuging:
      print("this in mediapipe")
      m = image.copy()
      top_left, bottom_right = left_hands_box_cordinations
      cx = int((top_left[0] + bottom_right[0]) / 2)
      cy = int((top_left[1] + bottom_right[1]) / 2)
      center = (cx, cy)
      cv2.circle(m, center, radius=5, color=(0, 0, 255), thickness=-1)
      cv2.rectangle(m, tuple(face_box_cordinations[0]), tuple(face_box_cordinations[1]), (255, 0, 0), 2)
      cv2.rectangle(m, tuple(left_hands_box_cordinations[0]), tuple(left_hands_box_cordinations[1]), (255, 0, 255), 2)
      cv2.rectangle(m, tuple(right_hands_box_cordinations[0]), tuple(right_hands_box_cordinations[1]), (255, 255, 0), 2)
      cv2.rectangle(m, tuple(pose_box_cordinations[0]), tuple(pose_box_cordinations[1]), (255, 255, 255), 2)
      print("this is tuple: ",tuple(pose_box_cordinations[0]), tuple(pose_box_cordinations[1]))
      # for i in range(len(pose_landmarks)):
      #     print("index: ",i,") pose: ",pose_landmarks[i])
      plt.imshow(m)
      plt.show()


    horizontal_hands_locations = []
    for hand in [left_hands_box_cordinations,right_hands_box_cordinations]:
      if hand[0][0]< pose_box_cordinations[0][0]:
        horizontal_hands_locations.append(1)
      elif hand[0][0] >= pose_box_cordinations[0][0] and hand[0][0] < face_box_cordinations[0][0]:
        horizontal_hands_locations.append(2)
      elif hand[0][0] >= face_box_cordinations[0][0] and hand[0][0] < face_box_cordinations[1][0]:
        horizontal_hands_locations.append(3)
      elif hand[0][0] >=face_box_cordinations[1][0] and hand[0][0]< pose_box_cordinations[1][0]:
        horizontal_hands_locations.append(4)
      else:
        horizontal_hands_locations.append(5)
    if self.debuging:
      self.info['horizontal'] = horizontal_hands_locations
      self.info['vertical'] = vertical_hands_locations
    return vertical_hands_locations,horizontal_hands_locations, right_hands_box_cordinations, pose_box_cordinations, face_box_cordinations, left_hands_box_cordinations

  def showFeatures(self):
    print("left hand distance from the face: ", self.info['left_distance'])
    print("right hand distance from the face: ", self.info['right_distance'])
    print("vertical hands regions: ",self.info["vertical"])
    print("horizontal hands regions: ",self.info["horizontal"])
    print("left hand image: ")
    plt.imshow(self.info["left_hand_image"])
    plt.show()
    print("right hand image: ")
    plt.imshow(self.info["right_hand_image"])
    plt.show()


In [9]:
class MediapipeFeaturesExtractor():
  def __init__(self,debuging =True,target_size = (512,512)):
    self.imageCropper = ImageCropper(debuging,target_size )
    self.featuresExtractorFromImage = FeaturesExtractorFromImage(debuging)
    self.debuging = debuging


  def extractFeaturesFromVideo(self, frames, landmarks, segmented_frames):
    cropped_images = []
    mediapipeFeatures = []
    updated_adjusted_landmarks = []
    previous_scaled_up_right_hand_landmarks = [0]
    previous_scaled_up_pose_landmarks = [0]
    previous_scaled_up_face_landmarks = [0]
    previous_scaled_up_left_hand_landmarks = [0]

    # for frame in tqdm(frames):
    #   self.extractor.extractLandmarks(frame)
    #   checkers.append(self.extractor.check())
    #   fframes.append(self.extractor.image)
    #   landmarks.append(self.extractor.get_x_y_landmarks_scaled_up())
    for frame,landmark,segmented_frame in zip(frames,landmarks, segmented_frames):

      checker = {
          "right_hand_landmarks":(landmark[-2][0]==1),
          "left_hand_landmarks":(landmark[-1][1]==1),
          "pose_landmarks":(landmark[-2][1]==1),
          "face_landmarks":(landmark[-1][0]==1),
      }
      if not checker['right_hand_landmarks'] and np.sum(previous_scaled_up_right_hand_landmarks) != 0 :
        landmark[:21,:] = previous_scaled_up_right_hand_landmarks

      if not checker['left_hand_landmarks'] and np.sum(previous_scaled_up_right_hand_landmarks) != 0 :
        landmark[532:-2,:] = previous_scaled_up_left_hand_landmarks

      if not checker['pose_landmarks'] and np.sum(previous_scaled_up_pose_landmarks) != 0:
        landmark[21:54,:]= previous_scaled_up_pose_landmarks
      if not checker['face_landmarks'] and np.sum(previous_scaled_up_face_landmarks) != 0:
        landmark[54:532,:]= previous_scaled_up_face_landmarks

      previous_scaled_up_right_hand_landmarks = landmark[:21,:]
      previous_scaled_up_pose_landmarks = landmark[21:54,:]
      previous_scaled_up_face_landmarks = landmark[54:532,:]
      previous_scaled_up_left_hand_landmarks = landmark[532:-2,:]

      cropped_image, _ = self.imageCropper.cropImage(segmented_frame, landmark[:-2], checker)
      new_frame, adjusted_landmark = self.imageCropper.cropImage(frame, landmark[:-2], checker)
      mediapipeFeature = self.featuresExtractorFromImage.extractFeatures(new_frame.permute(1,2,0).numpy().astype(dtype=np.uint16),checker,adjusted_landmark)

      updated_adjusted_landmarks.append(torch.tensor(adjusted_landmark))
      cropped_images.append(cropped_image)
      mediapipeFeatures.append(torch.tensor(mediapipeFeature))
      if self.debuging:
        self.showImages()
    return torch.stack(cropped_images), torch.stack(mediapipeFeatures), torch.stack(updated_adjusted_landmarks)

  def showImages(self):
    if self.debuging:
      self.imageCropper.showImage()
      self.featuresExtractorFromImage.showFeatures()
    else:
      print("sett debuging flag to True")


In [10]:
class ProcessVideo():
  def __init__(self,debuging=False,debuging_landmarks=False,target_size=(512,512)):
    self.mediapipeFeaturesExtractor = MediapipeFeaturesExtractor(debuging,target_size)
    self.extractor = Extractor(debuging, target_size)
    self.debuging = debuging

  def extractLandmarks(self, frames):
      landmarks, segmented_image, image = self.extractor.extractLandmarks(frames)
      return landmarks, segmented_image, image

  def extractFeaturesFromVideo(self, frames, landmarks, segmented_frames):
    cropped_images, mediapipeFeatures, updated_adjusted_landmarks = self.mediapipeFeaturesExtractor.extractFeaturesFromVideo(frames, landmarks, segmented_frames)
    return cropped_images, mediapipeFeatures, updated_adjusted_landmarks
 # ###

In [11]:
class Classifier():
    def __init__(self,prediciton_dictanory,model_weights_path=None,boundary_model_weights_path=None,num_partitions= 5,debuging=False):
       self.prediciton_dictanory = prediciton_dictanory
       self.model = Model().to(device)
       self.boundaryDetector = BoundaryDetector().to(device)

       if model_weights_path != None:
           if model_weights_path.endswith("checkpoint.pth"):
              if device ==torch.device('cpu'):
                  checkpoint = torch.load(model_weights_path, map_location=torch.device('cpu'))
                  self.model.load_state_dict(checkpoint["best_model_state_dict"])
              else:
                  checkpoint = torch.load(model_weights_path)
                  self.model.load_state_dict(checkpoint["best_model_state_dict"])
           else:
              self.model.load_state_dict(torch.load(model_weights_path))
       if boundary_model_weights_path != None:
           if device ==torch.device('cpu'):
              self.boundaryDetector.load_state_dict(torch.load(boundary_model_weights_path,map_location=torch.device('cpu')))
           else:
              self.boundaryDetector.load_state_dict(torch.load(boundary_model_weights_path))

       self.num_partitions= num_partitions
       self.debuging = debuging

    def detectBoundary(self,poseLandmarks, max_seq=120*6):
        train = list(poseLandmarks)
        numberofFrames = len(poseLandmarks)
        if numberofFrames < max_seq:
          remaining_frames = max_seq - numberofFrames
          train.extend(torch.zeros((remaining_frames,33,2)))
        train = torch.stack(train)
        train = train.to(device)
        output = self.boundaryDetector(train.unsqueeze(0))
        output = torch.argmax(output.squeeze(0), dim= -1)
        print(output)
        boundaries = [ii for ii,tt in enumerate(output) if tt==1]
        return boundaries
    ####
    def predict(self,cropped_images, mediapipefeaturess):
            self.model.eval()
        # if len(cropped_images.shape) == 5:
            seq_predictions = []
            for croped_image,mediapipefeatures in tqdm(zip(cropped_images,mediapipefeaturess)):
                # croped_image = torch.tensor(cropped_image)
                # mediapipefeatures = torch.tensor(mediapipefeatures)

                x_croped_image = []
                x_mediapipefeatures = []

                startingIndcies = torch.linspace(0,int(croped_image.shape[0]/2)-1,steps=self.num_partitions).long()

                for start in startingIndcies:
                    indcies = torch.linspace(start,int(croped_image.shape[0])-1,steps=32).long()
                    x_croped_image.append(croped_image[indcies])
                    x_mediapipefeatures.append(mediapipefeatures[indcies])

                x_croped_image = torch.stack(x_croped_image).to(device)
                x_mediapipefeatures = torch.stack(x_mediapipefeatures).to(device)
                if self.debuging:
                    for i in x_croped_image[0].cpu().permute(0,2,3,1):
                        plt.imshow(i/255)
                        plt.show()
                    print("this is the end of the video")
                # print("cropped images shape: ", x_croped_image.shape)
                # print("medipaipe features shape: ", x_mediapipefeatures.shape)
                with torch.no_grad():
                    mediapipefeatures = mediapipefeatures.float().to(device).unsqueeze(0)
                    croped_image = croped_image.float().to(device).unsqueeze(0)
                    probabilities = self.model(x_mediapipefeatures,x_croped_image)
                    softmax_probs = F.softmax(probabilities, dim=1)
                    top5_probs, top5_indices = torch.topk(softmax_probs, k=5, dim=1)

                    # Print top-5 for each prediction in the batch
                    for batch_idx in range(top5_indices.size(0)):
                        print(f"\nTop-5 predictions for sample {batch_idx}:")
                        for rank in range(5):
                            pred_idx = top5_indices[batch_idx][rank].item()
                            pred_word = self.prediciton_dictanory[pred_idx]
                            prob = top5_probs[batch_idx][rank].item()
                            print(f"  {rank+1}) {pred_word} - {prob*100:.2f}%")

                    # Still get argmax for majority voting
                    encdoed_output = torch.argmax(probabilities, dim=1)
                    predictions = [self.prediciton_dictanory[prediction.item()] for prediction in encdoed_output]

                    dataDictanory = {word:0 for word in set(predictions)}
                    for i in predictions:
                      dataDictanory[i]+=1
                    frequentLabel =predictions[0]
                    frequency = 0
                    for l,v in dataDictanory.items():
                      if v>frequency:
                        frequentLabel = l
                        frequency = v
                    seq_predictions.append(frequentLabel)
            return seq_predictions

        # else:
        #         x_croped_image = []
        #         x_mediapipefeatures = []


        #         startingIndcies = torch.linspace(0,int(croped_image.size(0)/2)-1,steps=self.num_partitions).long()

        #         for start in startingIndcies:
        #             indcies = torch.linspace(start,int(croped_image.size(0))-1,steps=32).long()
        #             x_croped_image.append(croped_image[indcies])
        #             x_mediapipefeatures.append(mediapipefeatures[indcies])

        #         x_croped_image = torch.stack(x_croped_image).to(device)
        #         x_mediapipefeatures = torch.stack(x_mediapipefeatures).to(device)
        #         # print("cropped images shape: ", x_croped_image.shape)
        #         # print("medipaipe features shape: ", x_mediapipefeatures.shape)
        #         with torch.no_grad():
        #             mediapipefeatures = mediapipefeatures.float().to(device).unsqueeze(0)
        #             croped_image = croped_image.float().to(device).unsqueeze(0)
        #             probabilities = self.model(x_mediapipefeatures,x_croped_image)
        #             encdoed_output = torch.argmax(probabilities,dim=1)
        #             predictions =[self.prediciton_dictanory[prediction.item()] for prediction in encdoed_output]
        #             dataDictanory = {word:0 for word in set(predictions)}
        #             for i in predictions:
        #               dataDictanory[i]+=1
        #             frequentLabel =predictions[0]
        #             frequency = 0
        #             for l,v in dataDictanory.items():
        #               if v>frequency:
        #                 frequentLabel = l
        #                 frequency = v

        #         return frequentLabel, predictions
# processVideo= ProcessVideo(True)
# frames = processVideo.extractFrames("/kaggle/input/new-video-for-testing/WIN_20250609_13_14_53_Pro.mp4")

# _=processVideo.extractMediapipeFeatures(frames)
# start = time.time()
# _,_,_ = processVideo.extractor.extractLandmarks(frames)
# time.time()-start
# Image.fromarray(np.array(frames,dtype= np.uint8))

In [12]:
class UploadManager():
    def __init__(self,prediciton_dictanory, model_weights_path=None,boundary_model_weights_path=None, num_partitions=5, extractOneFromEach= 2, intersectionRatio = 0.5, predict_after = 40,target_size=(512,512), debuging=False,debuging_landmarks=False,debuging_classifier= False):
        self.target_size = target_size
        self.extractOneFromEach = extractOneFromEach
        self.predict_after = predict_after
        self.intersectionRatio = intersectionRatio
        self.transform = transforms.CenterCrop((512, 512))

        self.mediapipeFeatures =[]
        self.cropedImages = []

        self.processVideo = ProcessVideo(debuging=debuging,debuging_landmarks=debuging_landmarks,target_size=(512,512))
        self.classifier = Classifier(prediciton_dictanory,model_weights_path=model_weights_path,boundary_model_weights_path= boundary_model_weights_path,num_partitions= 5,debuging= debuging_classifier)

    def checkLandmarks(self,frame):
        checker = self.processVideo.extractor.extractLandmarks(np.array(frame))
        return checker
    def extractFrames(self,video_path, video=False):
      frames = []
      valid_video=True
      cap = cv2.VideoCapture(video_path)
      fps = cap.get(cv2.CAP_PROP_FPS)
      if fps == 0:
        valid_video=False
        print("End of video or error occurred.")
        return None

      while True:
        ret, frame = cap.read()
        valid_video=True
        if not ret:
            break  # End of video

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # plt.imshow(frame)
        # plt.show()
        # Resize the frame to the target size before appending
        if not video:
            frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

        frame = Image.fromarray(frame)
        frame = self.transform(frame)
        # plt.imshow(frame)
        # plt.show()
        frames.append(frame)

      if valid_video:
        frames = np.array(frames, dtype=np.uint8)

        tensor = torch.tensor(frames)

        # transformed_images = [self.top_center_crop_cv2(image) for image in tensor]

        return tensor
      else:
        print("there is error in the frames")
        return None

    ###
    def predict(self, frames):
        landmarks, segmented_frames, frames =self.processVideo.extractLandmarks(np.array(frames))
        print("landmarks extracted successfully ")
        landmarks = np.array(landmarks)
        frames = np.array(frames)
        segmented_frames = np.array(segmented_frames)
        cropped_images, mediapipeFeatures, updated_adjusted_landmarks = self.processVideo.extractFeaturesFromVideo( frames, landmarks, segmented_frames)
        print("mediapipe features extracted successfully ")

        pose_landmarks = updated_adjusted_landmarks[:,21:54,:].clone().float()
        boundaries_dash = self.classifier.detectBoundary(pose_landmarks)
        print("boundaries is: ", boundaries_dash)

        boundaries = []
        for tt in range(len(boundaries_dash)):
          if tt!= 0 and boundaries_dash[tt]== boundaries_dash[tt-1]:
            continue
          else:
            boundaries.append(boundaries_dash[tt])

        def remove_consecutive_increases(lst):
            if not lst:
                return []

            result = [lst[0]]

            for i in range(1, len(lst)):
                if lst[i] != lst[i-1] + 1:
                    result.append(lst[i])

            return result



        boundaries = remove_consecutive_increases(boundaries)

        print(boundaries)

        print(cropped_images.shape)
        cropped_images_compo = []
        mediapipeFeatures_compo = []
        start = 0
        for end in boundaries:
            end = int(end)
            cropped_images_compo.append(cropped_images[start:end, :, :, :])
            mediapipeFeatures_compo.append(mediapipeFeatures[start:end, :])
            start = end
        cropped_images_compo = cropped_images_compo
        mediapipeFeatures_compo = mediapipeFeatures_compo
        print("generated sequences successfully ")

        predictions = self.classifier.predict(cropped_images_compo, mediapipeFeatures_compo)
        print("predictions successfully ")
        return predictions

 [43, 196, 197]

In [13]:
# uploadManager = UploadManager(
#         prediciton_dictanory={0: 'always',
#   1: 'apologize',
#   2: 'bad',
#   3: 'bring',
#   4: 'drink',
#   5: 'eat',
#   6: 'friend',
#   7: 'get_well',
#   8: 'good',
#   9: 'i',
#   10: 'never',
#   11: 'thanks',
#   12: 'time',
#   13: 'tomorrow',
#   14: 'wait',
#   15: 'where',
#   16: 'who',
#   17: 'why',
#   18: 'yesterday',
#   19: 'you'},
#         model_weights_path=model_path,
        # boundary_model_weights_path= boundaryModelPath,
        # num_partitions=5,
        # extractOneFromEach= 2,
        # intersectionRatio = 0.9,
        # predict_after = 60,
        # target_size=(512,512),
        # debuging=False,
        # debuging_landmarks=False,
        # debuging_classifier = False
        #     )

In [14]:
# frames =uploadManager.extractFrames("/content/you.mp4")
# # # frames =list(frames)
# # # frames.extend(frames)
# # # frames.extend(frames)
# # # frames = torch.stack(frames)
# print("number of frames: ",len(frames))
# prediction = uploadManager.predict(frames)
# prediction

In [15]:
# for i in frames[120:196]:
#     plt.imshow(i)
#     plt.show()

In [16]:
!pip install fastapi uvicorn nest_asyncio pyngrok python-multipart imageio[ffmpeg] imageio-ffmpeg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
from typing import List
import numpy as np
from matplotlib import pyplot as plt
import base64
import io
import tempfile
from fastapi import FastAPI, File, UploadFile, WebSocket
from fastapi.responses import JSONResponse


ngrok.kill()  # Clean up previous ngrok tunnels

nest_asyncio.apply()

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
NGROK_AUTH_TOKEN = "<YOUR_NGROK_AUTH_TOKEN>"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(8000)
print("This is Your URL: \n",public_url)

This is Your URL:                                                                                   
 NgrokTunnel: "https://b8ad-34-73-123-51.ngrok-free.app" -> "http://localhost:8000"


In [18]:
os.listdir("/kaggle/input/boundary-model")

['boundary_model_weights (2).pth']

In [ ]:
uploadManager = UploadManager(
        prediciton_dictanory={0: 'accident',
  1: 'always',
  2: 'apologize',
  3: 'bed',
  4: 'belt',
  5: 'breakfast',
  6: 'bring',
  7: 'forbidden',
  8: 'friend',
  9: 'full',
  10: 'get_well',
  11: 'glove',
  12: 'good',
  13: 'goodbye',
  14: 'hurry',
  15: 'police',
  16: 'same',
  17: 'sibling',
  18: 'single',
  19: 'thanks',
  20: 'time',
  21: 'tomorrow',
  22: 'wait',
  23: 'where',
  24: 'who',
  25: 'why'},
        model_weights_path=model_path,
        boundary_model_weights_path= boundaryModelPath,
        num_partitions=5,
        extractOneFromEach= 2,
        intersectionRatio = 0.9,
        predict_after = 60,
        target_size=(512,512),
        debuging=False,
        debuging_landmarks=False,
        debuging_classifier = False
            )
@app.post("/check")
async def upload_frame(frame: UploadFile = File(...)):
    print("im in: ")
    contents = await frame.read()
    image = Image.open(io.BytesIO(contents))
    print("this is the image: ",image)

    plt.imshow(image)
    plt.show()
    checker = uploadManager.checkLandmarks(image)


    return JSONResponse(content=checker, status_code=200)

@app.post("/upload")
async def upload_video(video: UploadFile = File(...)):

    os.makedirs("videos", exist_ok=True)

    video_path = f"videos/{video.filename}"
    with open(video_path, "wb") as buffer:
        shutil.copyfileobj(video.file, buffer)

    frames =uploadManager.extractFrames(video_path)
    # for frame in frames:
    #   plt.imshow(frame)
    #   plt.show()
    prediction = uploadManager.predict(frames)

    return {"status": "success", "glosses": prediction}

@app.post("/upload_desktop")
async def upload_video(video: UploadFile = File(...)):

    os.makedirs("videos", exist_ok=True)

    video_path = f"videos/{video.filename}"
    with open(video_path, "wb") as buffer:
        shutil.copyfileobj(video.file, buffer)

    frames =uploadManager.extractFrames(video_path,video=True)
    # for frame in frames:
    #   plt.imshow(frame)
    #   plt.show()
    prediction = uploadManager.predict(frames)

    return {"status": "success", "glosses": prediction}


uvicorn.run(app, host="0.0.0.0", port=8000)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751704511.942752     143 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751704511.981730     145 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751704511.986888     142 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751704512.023920     142 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751704512.036002     136 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751704512.085030     

landmarks extracted successfully 
mediapipe features extracted successfully 
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

1it [00:01,  1.93s/it]


Top-5 predictions for sample 0:
  1) thanks - 99.98%
  2) accident - 0.02%
  3) police - 0.00%
  4) get_well - 0.00%
  5) apologize - 0.00%

Top-5 predictions for sample 1:
  1) thanks - 99.98%
  2) accident - 0.02%
  3) get_well - 0.00%
  4) police - 0.00%
  5) apologize - 0.00%

Top-5 predictions for sample 2:
  1) accident - 77.60%
  2) thanks - 21.75%
  3) get_well - 0.36%
  4) where - 0.20%
  5) forbidden - 0.05%

Top-5 predictions for sample 3:
  1) thanks - 99.99%
  2) get_well - 0.01%
  3) accident - 0.00%
  4) forbidden - 0.00%
  5) where - 0.00%

Top-5 predictions for sample 4:
  1) thanks - 64.70%
  2) get_well - 29.37%
  3) accident - 5.50%
  4) why - 0.20%
  5) apologize - 0.07%
predictions successfully 
INFO:     154.189.139.66:0 - "POST /upload HTTP/1.1" 200 OK



100%|██████████| 57/57 [00:05<00:00, 10.81it/s]


...

landmarks extracted successfully 
mediapipe features extracted successfully 
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

0it [00:00, ?it/s]


Top-5 predictions for sample 0:


1it [00:01,  1.12s/it]

  1) friend - 99.92%
  2) time - 0.05%
  3) why - 0.03%
  4) hurry - 0.00%
  5) bed - 0.00%

Top-5 predictions for sample 1:
  1) why - 95.20%
  2) friend - 3.38%
  3) police - 1.10%
  4) belt - 0.16%
  5) time - 0.09%

Top-5 predictions for sample 2:
  1) full - 96.56%
  2) police - 3.32%
  3) tomorrow - 0.03%
  4) time - 0.02%
  5) belt - 0.02%

Top-5 predictions for sample 3:
  1) good - 100.00%
  2) bring - 0.00%
  3) single - 0.00%
  4) time - 0.00%
  5) tomorrow - 0.00%

Top-5 predictions for sample 4:
  1) good - 100.00%
  2) single - 0.00%
  3) time - 0.00%
  4) bring - 0.00%
  5) full - 0.00%
predictions successfully 
INFO:     154.189.139.66:0 - "POST /upload HTTP/1.1" 200 OK
